# Jobs titles and functions Dataset



Job functions and job titles are very different things. A job title is essentially
the name of a position within an organization filled by an employee. Job
function is the routine set of tasks or activities undertaken by a person in
that position. An employee's title and function are often closely related,
though not all job functions are clear based on title alone.


### So, in this notebook we are trying to build a system to recommend the job functions in which an employee with a specific job title can work



# 1. Data Engineering

In [17]:
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

 At first, We start by fetching and inspecting the dataset

In [18]:
#Inspecting the data
jobs_data = pd.read_csv("jobs_data.csv")
jobs_data.head(10)

,Unnamed: 0,title,jobFunction,industry
0,0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Marketing and Advertisi..."
1,1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa...",['Information Technology Services']
2,2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Computer Networking']"
3,3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme...","['Computer Software', 'Information Technology ..."
4,4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Information Technology ..."
5,5,Technical Support Engineer,"['IT/Software Development', 'Engineering - Tel...","['Information Technology Services', 'Computer ..."
6,6,Senior iOS Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Information Technology Services', 'Graphic D..."
7,7,Mechanical Engineer,['Engineering - Mechanical/Electrical'],"['Architectural and Design Services', 'Enginee..."
8,8,Real Estate Sales Specialist - 10th of Ramadan,['Sales/Retail'],['Real Estate/Property Management']
9,9,School Principal,"['Education/Teaching', 'Administration', 'Oper...",['Education']


As you can see above, We have to rename the Unnamed column to be the ID and the index of this dataset


In [19]:
jobs_data = jobs_data.rename(columns={"Unnamed: 0": "ID"})
jobs_data.set_index('ID')

,title,jobFunction,industry
ID,,,
0,Full Stack PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Marketing and Advertisi..."
1,CISCO Collaboration Specialist Engineer,"['Installation/Maintenance/Repair', 'IT/Softwa...",['Information Technology Services']
2,Senior Back End-PHP Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Computer Networking']"
3,UX Designer,"['Creative/Design/Art', 'IT/Software Developme...","['Computer Software', 'Information Technology ..."
4,Java Technical Lead,"['Engineering - Telecom/Technology', 'IT/Softw...","['Computer Software', 'Information Technology ..."
5,Technical Support Engineer,"['IT/Software Development', 'Engineering - Tel...","['Information Technology Services', 'Computer ..."
6,Senior iOS Developer,"['Engineering - Telecom/Technology', 'IT/Softw...","['Information Technology Services', 'Graphic D..."
7,Mechanical Engineer,['Engineering - Mechanical/Electrical'],"['Architectural and Design Services', 'Enginee..."
8,Real Estate Sales Specialist - 10th of Ramadan,['Sales/Retail'],['Real Estate/Property Management']


##### After inspecting the dataset, We noticed  2 repeated  job functions  which are :


 ['IT/Software Development', 'Engineering - Telecom/Technology']
 
#####  And

 ['Engineering - Telecom/Technology', 'IT/Software Development']
 
##### which are simillar to each other but inverted, So we decided to convert one to the other so they would be the exact same.
 

In [20]:
term = "['IT/Software Development', 'Engineering - Telecom/Technology']"
i=0
for job in jobs_data.jobFunction:

    if(job == term):
        jobs_data.jobFunction[i]=  "['Engineering - Telecom/Technology', 'IT/Software Development']"
    i+=1

Now we want to split the dataset into Training and testing datasets in 80%-20% respectively

In [21]:
jobs_data_train = jobs_data[0:8600]
jobs_data_test = jobs_data[8600:]

During inspecting the dataset, We have Also noticed that maybe morethan 95% of the __titles__ of all data instances that have
the __['Media/Journalism/Publishing', 'Marketing/PR/Advertising']__ term as a __JobFunction__ 
include at least 1 word from a list of keywords : __['marketing' , 'market', 'media' , 'digital' , 'social', 'advertising', 'brand','seo']__

<b>Tip:</b> This is why we decided to change all these titles into 1 unified Title : __"Marketing Specialist"__
and save all these Keywords in one list.


In [22]:
term = "['Media/Journalism/Publishing', 'Marketing/PR/Advertising']"
i=0
keywords = []
for jobFunction in jobs_data_train.jobFunction:  
    if(jobFunction == term): 
        jobs_data_train.title[i] = "Marketing Specialist"      
    i+=1

In [23]:
Marketing_Keywords = ['marketing' , 'market', 'media' , 'digital' , 'social', 'advertising', 'brand','seo']

Simillary, During inspecting the dataset, We have Also noticed that maybe morethan 95% of the __titles__ of all data instances that have
the __['Sales/Retail']__ term as a __JobFunction__ 
include at least 1 word from a list of keywords : __['sales' , 'real estate', 'real' , 'estate' , 'real-estate', 'broker', 'property']__

<b>Tip:</b> This is why we decided to change all these titles into 1 unified Title : __"Sales Consultant"__
and save all these Keywords in one list.


In [24]:
term = "['Sales/Retail']"
i=0
keywords = []
for jobFunction in jobs_data_train.jobFunction:  
    if(jobFunction == term): 
        jobs_data_train.title[i] = "Sales Consultant"      
    i+=1

In [25]:
Sales_Keywords = ['sales' , 'real estate', 'real' , 'estate' , 'real-estate', 'broker', 'property']

Here it's a little bit different, We found that there is a lot of data instances with a 
__jobFunction__ of __['Engineering - Telecom/Technology', 'IT/Software Development']__ but the titles of these instaces are very diverse and we can't just colect them manually like we did in the previous examples, So we had to do it automatically by appending all these titles into one list and then start to improve and enhance this list.

Also change each title into one Unified title : __Software Developer__


In [26]:
term = "['Engineering - Telecom/Technology', 'IT/Software Development']"
i=0
keywords = []
for jobFunction in jobs_data_train.jobFunction:
    
    if(jobFunction == term):
        words = jobs_data_train.title[i].split()  
        jobs_data_train.title[i]= "Software Developer"
        for word in words:
            if word not in keywords:
                word = word.lower()
                keywords.append(word)
    i+=1

##### From here We start working on the list of Software kewords

In [27]:
keywords_noDupl = list(dict.fromkeys(keywords))
keywords_noDupl

['full',
 'stack',
 'php',
 'developer',
 'senior',
 'back',
 'end-php',
 'java',
 'technical',
 'lead',
 'ios',
 'full-stack',
 '-',
 'joomla',
 'expert',
 'website',
 'front-end',
 'back-end',
 'odoo',
 'software',
 'erp',
 'implementer',
 'alexandria',
 '.net',
 'tech',
 'android',
 'team',
 'leader',
 'mid',
 'web',
 'associate',
 'engineer',
 'wordpress',
 'junior',
 '(',
 'angular',
 '4+',
 ')',
 'tester',
 '"laravel"',
 'mobile',
 'android/ios',
 'front',
 'end',
 'vue.js',
 '/',
 'ui/ux',
 'director',
 'in',
 'tunisia',
 'ge',
 'mvc',
 'oracle',
 'database',
 'admin',
 'flutter',
 'nasr',
 'city',
 'it',
 'help',
 'desk',
 'specialist',
 'minya',
 'nodejs',
 'backend',
 'rpa',
 'asp.net',
 'crm',
 'frontend',
 '(jquery)',
 'internship',
 '"laravel',
 'project"',
 'application',
 'programmer',
 'monitoring',
 'agent',
 'data',
 'etl',
 'security',
 'business',
 'analyst',
 'core',
 'rest',
 'to',
 'graphql',
 'project',
 'bi/bw',
 'consultant',
 'architect',
 'ionic',
 'app',
 '

1. Remove Spaces, punctuations, numbers and single charcter using __regular expressions__
2. Manullay inspect and collect some generic words inside the list to be removed
3. Add 2 parts words that are removd with generic words

In [28]:
import re
i=0
for word in keywords_noDupl:
    # Removing multiple spaces
    word = re.sub(r'\s+', ' ', word)
    # Remove punctuations and numbers
    word = re.sub('[^a-zA-Z]', ' ', word)
    # Single character removal
    word = re.sub(r"\s+[a-zA-Z]\s+", ' ', word)

    keywords_noDupl[i]=word
    i+=1
    
GeneralWords = ['full','alexandria','owner', 'surveillance','access', 'system','leader','stack','senior','back','lead','-','expert','team', 'leader', 'mid', 'associate', 'engineer', 'end',  'front', 'nasr', 'city', 'help', 'desk', 'specialist','minya', 'internship','project',  'monitoring','agent', 'security',  'senior\\team','#','freelance/part','consultant','architect','egypt', 'solution','4+','brazil', 'saudi', 'arabia', 'owner','kickstart','[marketing]', 'designer','riyadh','alex','ksa','riseup', 'contact', 'salesforce.com', 'dubai', 'business', 'analyst', 'core', 'rest','to', 'benha', 'product', 'principal', 'indonesia', 'implementation', '(sales', 'buzz)', 'intern', 'cross', 'remotely', 'platform','infrastructure','cairo','liferay', 'turkey','-entity', 'professional', 'bi','host','sharqia','tunis','(part', 'department)','only','india', 'time)','forms','officer', 'success','bw','fresh','graduates', 'monufya','coordinator','e-commerce', 'dakahlia','graduate','unpaid','(outsource)', 'hr','writer','jeddah','services','subject','mm', 'middle', 'next','on','up','deep','learning','pre-sales','ware','head','level','of','delivery','(leading','branch)','factor','stack/', 'supervisor', 'design','unified','banking', 'mansoura','communications', 'manager', 'functional', 'consultant)', '(r&d)', 'scientist','hurghada','(alexandria)','media','(information']
i=0

 

for word in keywords_noDupl:
    if word in GeneralWords:
        keywords_noDupl.remove(word)
    i+=1
#add 2 parts words technical lead, tech lead, back end, front end,react native, computer engineer, c#,middle ware, ruby on rails

keywords_noDupl.append('technical lead')
keywords_noDupl.append('back end')
keywords_noDupl.append('front end')
keywords_noDupl.append('react native')
keywords_noDupl.append('computer engineer')
keywords_noDupl.append('c#')
keywords_noDupl.append('middle ware')
keywords_noDupl.append('ruby on rails')


As a last step, We want to remove the English stopping words from the list, So we will be using Natural language dictionary

__By trial and error, In  order to eliminate ALL Stopping words from the list we need to run this cell three times__

In [35]:
#in order to include stopwords in our code we had to use nltk.download() which will open a window for us,
#from which we can choose stopwords to download 

# nltk.download()
import nltk
from nltk.corpus import stopwords

for i in range (0,3):
    for word in keywords_noDupl:
        if word in (stopwords.words('english')):
            keywords_noDupl.remove(word)
        
#remove remaining spaces
k=0
for word in keywords_noDupl:
    if word == ' ':
        del keywords_noDupl[k]
    k+=1

Software_Keywords = keywords_noDupl
Software_Keywords

['stack',
 'php',
 'developer',
 'back',
 'end php',
 'java',
 'technical',
 'ios',
 'full stack',
 'joomla',
 'website',
 'front end',
 'back end',
 'odoo',
 'software',
 'erp',
 'implementer',
 ' net',
 'tech',
 'android',
 'leader',
 'web',
 'engineer',
 'wordpress',
 'junior',
 'angular',
 '  ',
 'tester',
 ' laravel ',
 'mobile',
 'android ios',
 'end',
 'vue js',
 'ui ux',
 'director',
 'tunisia',
 'ge',
 'mvc',
 'oracle',
 'database',
 'admin',
 'flutter',
 'city',
 'desk',
 'minya',
 'nodejs',
 'backend',
 'rpa',
 'asp net',
 'crm',
 'frontend',
 ' jquery ',
 ' laravel',
 'project ',
 'application',
 'programmer',
 'agent',
 'data',
 'etl',
 'business',
 'core',
 'graphql',
 'bi bw',
 'architect',
 'ionic',
 'app',
 ' ionic',
 'flutter ',
 'node js',
 'fullstack',
 'ms',
 'dynamics',
 'react',
 'native',
 'epm',
 'visual',
 'basic',
 'sharepoint',
 'sap',
 'fi co',
 'cisco',
 'integration',
 'ucce',
 ' freelance ',
 'ios android',
 ' flutter ',
 'part',
 'time',
 'cross platfor

# 2. Machine learning Model

At first we split the training data into 2 parts, __features and targets__

After inspecting the data we Decided that only the titles will matter to us during training and not the industry because it doesn't change that much in the job function of each job title

In [36]:
labels = jobs_data_train['jobFunction']
train_data = jobs_data_train['title']


As we are dealing with text dataset, We first want to vectorize this dataset because Machine learning models only deal with numbers. 

So constructing a pipleline including the vectorizer and the Machine learning will be the best choice.


According to some researches and articles, Multinomaial Naive Bayes is one if the best model to be used with text dataset, However we wanted to inspect 2 other classifier to make sure of there hypothesis

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf',  MultinomialNB()),
              ])
nb.fit(train_data, labels)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In order to test our testing dataset accordig to the approach of Keywords list we have followed, We need to follow some steps

1. We have to inspect each title in the dataset,
2. check whether it includes any words that belongs to any of the the 3 lists,
3. if yes, change the title into the assigned title of the List, otherwise don't change it
4. apply the model on the final result

In [38]:

i=8600
for title in jobs_data_test.title:
    Software_exist = False 
    Sales_exist = False
    Marketing_exist = False
    
    words = title.split()
    for word in words:
        word = word.lower()
        
        if word in Sales_Keywords:
            Sales_exist = True
            break
        if word in Marketing_Keywords:
            Marketing_exist =True
            break
        if word in Software_Keywords:
            Software_exist =True
            break

        
    if Software_exist:    
        jobs_data_test.title[i]= "Software Developer"
    if Sales_exist:
            jobs_data_test.title[i]= "Sales Consultant"
    if Marketing_exist:
            jobs_data_test.title[i]= "Marketing Specialist"


    
    
    i+=1

In [39]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

test_labels = jobs_data_test['jobFunction']
test_data = jobs_data_test['title']

y_pred = nb.predict(test_data)

print('accuracy %s' % accuracy_score(test_labels,y_pred))

accuracy 0.42555066079295156


After trying the 3 models:
1. Multinomial Naive bayes
2. SGD classifier
3. Logisitic regression

We found that Multinomial Naive bayes is the best classifier we can use 